In [70]:
import sys
sys.path.append('/content/FreewayGame/leonardo/')
%matplotlib inline
from collections import defaultdict
from typing import List

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots

In [71]:
def print_result(i, scores, total_reward, score):
    if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

In [88]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

In [154]:
def sampler(n_runs=5000):
    env, initial_state = environment.get_env()
    observations = []
    agent = agents.Baseline()
    for i in tqdm(range(n_runs)):
        game_over = False
        state = env.reset()
        
        state = reduce_state(state)[RAM_mask].data.tobytes()
        action = agent.act(state)

        score = 0
        total_reward = 0

        while not game_over:
            
            old_state = state
            old_action = action
            ob, reward, game_over, _ = env.step(action)

            ob = reduce_state(ob)
            
            observations.append(list(ob[RAM_mask]))
            reward = reward_policy(reward, ob, action)

            total_reward += reward

            if reward == reward_policy.REWARD_IF_CROSS:
                score += 1

            state = ob[RAM_mask].data.tobytes()

            action = agent.act(state)  # Next action

    return observations

In [155]:
obs = sampler(1)

100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


In [172]:
obs[80]

[2, 0, 16, 20, 26, 0, 26, 0, 13, 26, 0, 0]

In [177]:
asd = [2, 0, 16, 20, 26, 0, 26, 0, 13, 26, 0, ]
asd[-5:] = [f if f>17 else 0 for f in obs[80][-5:]]

In [175]:

[f if f<20 else 0 for f in obs[80][-10:-5]]

[16, 0, 0, 0, 0]

In [163]:
obs[:100]


[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [9, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
 [9, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0],
 [10, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0],
 [12, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0],
 [13, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0],
 [14, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 6, 13, 0, 0, 0, 0, 0],
 [17, 0, 0, 0, 0, 7, 14, 0, 0, 0, 0, 0],
 [17, 0, 0, 0, 0, 7, 15, 0, 0, 0, 0, 0],
 [18, 0, 0, 0, 0, 8, 16, 0, 0, 0, 0, 0],
 [19, 0, 0, 0, 0, 8, 17, 0, 0, 0, 0, 0],
 [20, 0, 0, 0, 0, 9, 18, 0, 0, 0, 0, 0],
 [21, 0, 0, 0, 6, 9, 19, 0, 0, 0, 0, 0],
 [22, 0, 0, 0, 6, 10, 20, 0, 0, 0, 0, 0],
 [23, 0, 0, 0, 7, 10, 20, 0, 0, 0, 0, 0],
 [24, 0, 0, 0, 7, 11, 21, 0, 0, 0, 0, 0],
 [25, 0, 0, 0, 7, 11, 22, 0, 0, 0, 0, 0],
 [25, 0, 0, 0, 8, 11, 23, 0, 0

In [168]:
[f for f in obs[80][-5:]]

[0, 13, 26, 0, 0]

In [148]:
d = defaultdict(set)
for o in obs:
    for i, r in enumerate(RAM_mask):
        d[r].add(o[i])
        

In [149]:
d

defaultdict(set,
            {14: {6,
              7,
              8,
              9,
              10,
              11,
              12,
              13,
              14,
              15,
              16,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              26,
              27,
              28,
              29,
              30,
              31,
              32,
              33,
              34,
              35,
              36,
              37,
              38,
              39,
              40,
              41,
              42,
              43,
              44,
              45,
              46,
              47,
              48,
              49,
              50,
              51,
              52,
              53,
              54,
              55,
              56,
              57,
              58,
              59,
           

In [26]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3
    return ob

def reduce_state2(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3
    return ob
    
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

def train(agent, env, n_runs=5000, n_render=500, n_print=10):
    scores = []
    total_rewards = []
    for i in range(n_runs):
        render = i % n_render == (n_render-1)

        E = defaultdict(lambda: np.zeros(2))

        game_over = False
        state = env.reset()
        state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
        action = agent.act(state)

        score = 0
        total_reward = 0

        while not game_over:
            if render:
                time.sleep(0.005)
                env.render()

            old_state = state
            old_action = action
            ob, reward, game_over, _ = env.step(action)

            ob = reduce_state(ob)
            reward = reward_policy(reward, ob, action)

            total_reward += reward

            if reward == reward_policy.REWARD_IF_CROSS:
                score += 1

            state = ob[RAM_mask].data.tobytes()

            action = agent.act(state)  # Next action

            E[old_state][old_action] += 1

            agent.update(old_s=old_state, new_s=state, old_a=old_action, new_a=action, reward=reward, E=i+1)

        scores.append(score)
        total_rewards.append(total_reward)

        if i % n_print == 0:
            print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")
    return scores, total_rewards
    

    
    

In [29]:
def run_lfa_experiment(n_runs=5000, AVAILABLE_ACTIONS = 2, GAMMA = 0.99, N0 = 2.5, ALPHA = 0.001,
                        LAMB = 0.0001, REWARD_IF_CROSS=500, REWARD_IF_COLISION=-10, REWARD_IF_STILL=-1):

    env, initial_state = environment.get_env()
    
    reward_policy.REWARD_IF_CROSS = REWARD_IF_CROSS
    reward_policy.REWARD_IF_COLISION = REWARD_IF_COLISION
    reward_policy.REWARD_IF_STILL = REWARD_IF_STILL

    agent = agents.SarsaLFA(gamma=GAMMA, state_size=len(RAM_mask), available_actions=AVAILABLE_ACTIONS, N0=N0, alpha=ALPHA, lamb=LAMB)
    agent.trainScaler(env, RAM_mask,)
    
    scores = train(agent, env, n_runs=n_runs, n_render=500, n_print=10)
    aux_plots.plot_scores(scores)

In [30]:
%%time
run_lfa_experiment()

1
1
0
0
0
1
1
1
0
0
0
1
0
0
1
0
1
0
0
0
1
0
1
1
0
1
1
1
1
1
1
0
0
0
1
1
1
1
1
0
0
1
0
1
1
1
0
0
1
1
1
1
0
1
0
1
1
0
0
0
1
0
0
0
0
1
0
0
0
1
1
1
0
1
0
0
0
1
0
1
1
1
0
0
1
0
0
0
1
1
0
1
1
1
0
1
1
0
1
1
1
0
0
0
1
0
0
0
0
1
1
0
0
0
1
1
1
1
0
0
1
1
1
0
1
1
1
0
0
1
0
1
1
1
1
0
1
0
0
1
1
0
1
0
1
1
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
1
0
0
0
0
0
1
1
1
1
1
1
0
0
1
0
0
1
1
1
1
0
1
0
1
1
0
0
1
0
1
0
0
0
1
0
0
0
0
1
1
1
0
1
1
1
1
0
0
1
0
1
1
0
1
1
1
1
1
0
1
0
1
0
0
0
1
0
1
1
0
0
0
1
1
1
0
1
1
1
1
1
0
0
1
0
1
1
1
0
1
0
1
1
1
0
0
1
1
0
0
0
1
1
1
0
0
0
1
0
1
1
0
0
0
1
0
0
0
1
1
1
1
0
0
1
0
0
1
0
0
0
1
1
0
0
0
1
0
1
1
1
0
0
1
1
1
1
0
1
0
0
0
1
1
1
0
1
1
0
1
0
1
1
1
0
0
1
0
1
1
1
0
0
1
1
1
0
0
1
1
1
0
0
0
0
1
0
0
0
0
1
1
0
0
1
0
1
0
1
1
0
0
0
1
1
1
0
1
0
1
1
1
0
1
1
0
0
0
1
0
0
1
1
1
0
0
0
1
0
0
1
1
1
1
1
1
0
0
1
0
0
1
0
1
0
1
0
0
1
0
0
1
0
0
0
1
0
1
0
1
0
0
1
0
1
0
1
0
0
0
1
0
0
1
1
0
1
1
1
0
0
0
0
1
1
1
0
0
1
1
1
1
0
0
0
0
1
0
0
1
0
0
0
0
1
1
1
0
1
0
0
1
1
1
1
0
1
0
1
0
1
1
0
0
1
1
0
0
1
1
0
1
1


KeyboardInterrupt: 